In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option("display.max_columns",200)

In [ ]:
data_train=pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv',low_memory=False)

In [ ]:
data_train.head()

In [ ]:
data_train.columns

In [ ]:
data_train.shape

In [ ]:
data_train.dtypes

In [ ]:
data_train['title'].isnull().sum()
data_train['title']=data_train['title'].drop_duplicates()

In [ ]:
data_train=data_train[[#'adult', 
            #'belongs_to_collection', 
            #'budget', 
            #'genres', 
            #'homepage', 'id',
       #'imdb_id', 
        #'original_language', 
            #'original_title', 
        'overview',
       #'popularity', 'poster_path', 'production_companies',
       #'production_countries', 'release_date', 'revenue', 'runtime',
       #'spoken_languages', 'status', 
           'tagline','title'
      #, 'video',
       #'vote_average', 'vote_count'
           ]]

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train['overview'].fillna('',inplace=True)

In [ ]:
data_train['tagline'].fillna('',inplace=True)

In [ ]:
data_train['title'].fillna('',inplace=True)

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train['overall']=data_train['title']+" "+data_train['overview']+" "+data_train['tagline']

In [ ]:
data_train['overall']=data_train['overall'].str.lower()

In [ ]:
data_train['overall']

In [ ]:
import re # regular expression
import string


def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

data_train['overall']= data_train['overall'].apply(remove_URL)
data_train['overall']= data_train['overall'].apply(remove_html)
data_train['overall']= data_train['overall'].apply(remove_emoji)
data_train['overall']= data_train['overall'].apply(remove_punct)

In [ ]:
data_train['overall']

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
def text_preprocessing(text):
    doc=nlp(text)
    pre=[]
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        pre.append(token.lemma_)
    return " ".join(pre)

In [ ]:
data_train['overall']=data_train['overall'].apply(text_preprocessing)

In [ ]:
data_train['overall']

In [ ]:
data_train['overall'].isnull().sum()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
tfidf_matrix=tf.fit_transform(data_train['overall'])

In [ ]:
tfidf_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
title=data_train['title'].tolist()
index={title[i]:i for i in range(len(title))}

In [ ]:
#user=input("Enter Movie Name:")
user="Iron Man"
movie_index = index.get(user)
if movie_index is None:
    print("Sorry The Movie you provided is not present in our current dataset..")
else:
    sim_score=cosine_similarities[movie_index]
    sim_moive_indx=np.argsort(sim_score)[::-1][1:5+1]
    for i in sim_moive_indx:
        print(title[i])